In [14]:
from main import ee, feats
print(feats.keys())

dict_keys(['Tailing', 'Processing_Facilities', 'Dumps', 'Active_Mines', 'Artisinal_Mining', 'Smelters', 'Exploration'])


In [19]:
import ipyleaflet
from ipywidgets import HTML

# Setup interactive map
ee_map = ipyleaflet.Map(zoom=3, layout={'height':'600px'}, scroll_wheel_zoom=True)

def GetTileLayerUrl(ee_image_object):
    """Load tile url from EE for a given image object"""
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

def ResetLayers(map_object):
    """Clear map overlays"""
    for lyr in map_object.layers[1:]:
        map_object.remove_layer(lyr)

def AddTileLayer(map_object, ee_image_object):
    """Add EE tile overlay"""
    tile_url = GetTileLayerUrl(ee_image_object)
    map_object.add_layer(ipyleaflet.TileLayer(url=tile_url))

def AddPointsLayer(map_object, feats):
    """Add point features overlay"""
    markers = []
    geojson = feats.getInfo()
    for feat in geojson['features']:
        geo = feat['geometry']
        if geo['type'] != 'Point': continue
        msg = HTML()
        msg.value = feat['properties']['name']
        msg.description = feat['properties']['description']
        lng, lat = geo['coordinates']
        marker = ipyleaflet.Marker(location=(lat, lng), draggable=False)
        marker.popup = msg
        markers.append(marker)
    marker_cluster = ipyleaflet.MarkerCluster(
        markers=markers
    )
    map_object.add_layer(marker_cluster) 

In [16]:
# Using Landsat 8 Surface Reflectance Tier 1
# Resolution of 30m^2
# <https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR>
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

# Rename band names
# (see prev link for reference)
band_names = {
    'B1': 'ultra_blue',
    'B2': 'blue',
    'B3': 'green',
    'B4': 'red',
    'B5': 'nir',
    'B6': 'swir_1',
    'B7': 'swir_2',
    
    # Not in this dataset
    #'B8': 'pan',
    #'B9': 'cirrus',
    
    'B10': 'tirs_1',
    'B11': 'tirs_2',
    'sr_aerosol': 'sr_aerosol', 
    'pixel_qa': 'pixel_qa',
    'radsat_qa': 'radsat_qa'
}
old_names = ee.List(list(band_names.keys()))
new_names = ee.List(list(band_names.values()))
l8sr = l8sr.select(old_names, new_names)

In [17]:
def maskClouds(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

In [20]:
# Visualize points
ResetLayers(ee_map)
filtered = (
    l8sr.filterDate('2016-01-01', '2017-01-01').map(maskClouds)
        .median() # or mean, max, min
    
        # min=value to map to RGB8 value 0
        # max=value to map to RGB8 value 255
        .visualize(min=0, max=3000, bands=['red', 'green', 'blue'])
)
AddTileLayer(ee_map, filtered)
AddPointsLayer(ee_map, feats['Active_Mines'])
ee_map

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

# Classification

- <https://developers.google.com/earth-engine/classification>

In [ ]:
# Set class labels
feats = {k: fc.map(lambda f: f.set('class', i)) for i, (k, fc) in enumerate(feats.items())}

# Num examples
sum(fc.size().getInfo() for fc in feats.values())

In [ ]:
# Datasets
fc1 = feats['Active_Mines'].map(lambda f: f.set('class', 0))
fc2 = feats['Processing_Facilities'].map(lambda f: f.set('class', 1))
points = fc1.merge(fc2)
points.size().getInfo()

In [ ]:
# Get image collection w/in the bounds of these points
ic = l8sr.filter(ee.Filter.geometry(points.geometry()))
ic.size().getInfo()

In [ ]:
# Reduce image
im = ic.median()

In [ ]:
im_viz = im.visualize(min=0, max=3000, bands=['B4', 'B3', 'B2'])
ResetLayers(ee_map)
AddTileLayer(ee_map, im_viz)
ee_map

In [ ]:
# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']
im = im.select(bands)

# Overlay the points on the imagery to get training.
training = im.sampleRegions(collection=points, properties=['class'], scale=30)

# Train a CART classifier with default parameters.
trained = ee.Classifier.cart().train(training, 'class', bands)

# Classify the image with the same bands used for training.
classified = image.select(bands).classify(trained)

In [ ]:
cl_viz = classified.visualize(min=0, max=1, bands=['classification'], palette=['00FF00', 'FF0000'])
ResetLayers(ee_map)
AddTileLayer(ee_map, cl_viz)
ee_map

---

# Learning

In [ ]:
# "Landsat 8 image collection
# which has been processed to surface reflectance values
# (i.e. the processing has attempted to remove the atmospheric effects)."
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

sample_image = ee.Image(
    l8sr.filterDate('2017-11-01', '2017-12-01')
        .filterBounds(ee.Geometry.Point(-73.9957, 40.7262))
        .first()
)
band_names_original = sample_image.bandNames()

l8_bands = ee.Dictionary({
    'B1': 'ultra_blue',
    'B2': 'blue',
    'B3': 'green',
    'B4': 'red',
    'B5': 'nir',
    'B6': 'swir_1',
    'B7': 'swir_2',
    'B8': 'pan',
    'B9': 'cirrus',
    'B10': 'tirs_1',
    'B11': 'tirs_2',
    'sr_aerosol': 'sr_aerosol', 
    'pixel_qa': 'pixel_qa',
    'radsat_qa': 'radsat_qa'
})
band_names_new = l8_bands.values(sample_image.bandNames())
l8sr = l8sr.select(band_names_original, band_names_new)

In [ ]:
ResetLayers(ee_map)

In [ ]:
filtered = (
    l8sr.filterDate('2017-01-01', '2017-01-02')
        .median()
#         .mean()
#         .max()
#         .min()
#         .reduce(ee.Reducer.percentile([25])).rename(band_names_new)
        .visualize(min=0, max=3000, bands=['red', 'green', 'blue'])
)
AddTileLayer(ee_map, filtered)

# References

- <https://github.com/tylere/PyDataNYC2017/blob/master/ipynb/satellite_analysis.ipynb>
- <https://github.com/tylere/ee-jupyter-examples/blob/master/2%20-%20EE%20101.ipynb>
- <https://ipyleaflet.readthedocs.io/en/latest/api_reference/map.html>
- <https://github.com/tylere/ee-jupyter-examples/blob/master/4%20-%20Earth%20Engine%20and%20TensorFlow.ipynb>
- <https://developers.google.com/earth-engine/api_docs>